In [ ]:
# pip install keras-tuner --upgrade
import numpy as np
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow import keras
from tensorflow.keras import models
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import  Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout, Resizing
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import keras_tuner
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt

%matplotlib inline

# reproducibility
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
dataset_dir = '/kaggle/input/anndl-challenge2/'

x = np.load(dataset_dir + 'x_train.npy')
y = np.load(dataset_dir + 'y_train.npy')
y = tfk.utils.to_categorical(y)

input_shape = x.shape[1:]
classes = y.shape[-1]
batch_size = 128
epochs = 200
print ('input shape:',input_shape,', classes:', classes)

input shape: (36, 6) , classes: 12


In [ ]:
# scaler normalization
X = x.reshape(-1,x.shape[-1])
X_scaled = RobustScaler().fit(X).transform(X)
xs = X_scaled.reshape(x.shape)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(xs, y, test_size=.2)

#### Define model(s)

In [ ]:
def model_skeleton(input_shape, classes,
                   modelchoice,
                   units1,
                   units2,
                   units3,
                   unitsD1,
                   unitsD2,
                   dense2,
                   dropout1,
                   dropout2,
                   dropout3,
                   dropout4,
                   poolchoice1,
                   poolchoice2,
                   poolchoice3,
                   avgpool,
                   lr):
    # Build the neural network layer by layer
    model = tfk.models.Sequential()
    model.add(tfkl.Input(shape=input_shape, name='Input'))

    # Feature extractor v1: LSTM
    if modelchoice in [1,2,3]:
        model.add(tfkl.LSTM(units1, return_sequences=True))
        if dropout1:
            model.add(tfkl.Dropout(.5, seed=seed))
        if modelchoice>1:
            model.add(tfkl.LSTM(units2))#(layer if 'layer' in locals() else input_layer)
            if dropout2:
                model.add(tfkl.Dropout(.3, seed=seed))
        if modelchoice>2:
            model.add(tfkl.LSTM(units3))#(layer if 'layer' in locals() else input_layer)
            if dropout3:
                model.add(tfkl.Dropout(.2, seed=seed))
                
    # Feature extractor v2: Bidirectional LSTM
    elif modelchoice in [4,5,6]:
        model.add(tfkl.Bidirectional(tfkl.LSTM(units1, return_sequences=True)))
        if dropout1:
            model.add(tfkl.Dropout(.5, seed=seed))
        if modelchoice>4:
            model.add(tfkl.Bidirectional(tfkl.LSTM(units2)))#(layer if 'layer' in locals() else input_layer)
            if dropout2:
                model.add(tfkl.Dropout(.3, seed=seed))
        if modelchoice>5:
            model.add(tfkl.Bidirectional(tfkl.LSTM(units3)))#(layer if 'layer' in locals() else input_layer)
            if dropout3:
                model.add(tfkl.Dropout(.2, seed=seed))
    
    # Feature extractor v3: Conv1D
    if modelchoice in [7,8,9]:
        model.add(tfkl.Conv1D(units1, 3, padding='same', activation='relu'))
        if poolchoice1:
            model.add(tfkl.MaxPooling1D())
        if dropout1:
            model.add(tfkl.Dropout(.5, seed=seed))
            
        if modelchoice>7:
            model.add(tfkl.Conv1D(units2, 3, padding='same', activation='relu'))#(layer if 'layer' in locals() else input_layer)
            if poolchoice2:
                model.add(tfkl.MaxPooling1D())
            if dropout2:
                model.add(tfkl.Dropout(.3, seed=seed))
                
        if modelchoice>8:
            model.add(tfkl.Conv1D(units3, 3, padding='same', activation='relu'))#(layer if 'layer' in locals() else input_layer)
            if poolchoice3:
                model.add(tfkl.MaxPooling1D())
            if dropout3:
                model.add(tfkl.Dropout(.2, seed=seed))
        if avgpool:
            model.add(tfkl.GlobalAveragePooling1D())

    if dropout4:
        model.add(tfkl.Dropout(.3, seed=seed))

    # Classifier: always at least 1 dense
    model.add(tfkl.Dense(unitsD1, activation='relu'))
    if dense2: #maybe 2 denses
        model.add(tfkl.Dense(unitsD2, activation='relu'))
        
    # Output
    model.add(tfkl.Flatten())
    model.add(tfkl.Dense(classes, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=tfk.optimizers.Adam(learning_rate=lr),
                  metrics='accuracy')

    # Return the model
    return model

In [ ]:
def build_model(hp):
    modelchoice = hp.Choice("modelchoice", [7,8,9]) # 1-3:lstm, 4-6:bilstm, 7-9:conv1d
    poolchoice1 = hp.Boolean("poolchoice1")
    poolchoice2 = hp.Boolean("poolchoice2")
    poolchoice3 = hp.Boolean("poolchoice3")
    avgpool = hp.Boolean("avgpool")
    units1 = hp.Int("units1", min_value=32, max_value=1024, step=32)
    units2 = hp.Int("units2", min_value=32, max_value=512, step=32)
    units3 = hp.Int("units3", min_value=32, max_value=256, step=32)
    unitsD1 = hp.Int("unitsD1", min_value=32, max_value=1024, step=32)
    unitsD2 = hp.Int("unitsD2", min_value=32, max_value=512, step=32)
    dense2 = hp.Boolean("dense2")
    dropout1 = hp.Boolean("dropout1")
    dropout2 = hp.Boolean("dropout2")
    dropout3 = hp.Boolean("dropout3")
    dropout4 = hp.Boolean("dropout4")
    lr = hp.Float("lr", min_value=1e-6, max_value=1e-3, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = model_skeleton(input_shape, classes,
                          modelchoice=modelchoice,
                          units1=units1,
                          units2=units2,
                          units3=units3,
                          unitsD1=unitsD1,
                          unitsD2=unitsD2,
                          dense2=dense2,
                          dropout1=dropout1,
                          dropout2=dropout2,
                          dropout3=dropout3,
                          dropout4=dropout4,
                          poolchoice1=poolchoice1,
                          poolchoice2=poolchoice2,
                          poolchoice3=poolchoice3,
                          avgpool=avgpool,
                          lr=lr)
    return model

build_model(keras_tuner.HyperParameters())

tuner = keras_tuner.Hyperband(
    build_model,
    objective='val_accuracy',
#     max_trials=50, # different combinations of parameters #useless for hyperband
    overwrite=True,
    executions_per_trial=5, # try to retrain same model
    directory="HPT",
    project_name="LSTM_HPT",
    )

2022-12-16 14:12:06.109407: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 14:12:06.110674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 14:12:06.404866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 14:12:06.405849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 14:12:06.406729: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

#### Run the tuner

In [ ]:
tuner.search(x_train, y_train,
             validation_data=(x_val,y_val),
             epochs=15,
             callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=5, factor=0.5, min_lr=1e-5)
             ])

Trial 254 Complete [00h 02m 13s]
val_accuracy: 0.5353909432888031

Best val_accuracy So Far: 0.7304526567459106
Total elapsed time: 01h 50m 21s


#### Elaborate results

In [ ]:
model_list = tuner.get_best_models(num_models = 2)
best_model = model_list[0]

best_model.build()
best_model.summary()

Model loaded!

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 36, 6)]           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 36, 512)           9728      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 18, 512)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 18, 512)           786944    
                                                                 
 global_average_pooling1d_1   (None, 512)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 512) 

In [ ]:
tuner.results_summary()

Results summary
Results in HPT/LSTM_HPT
Showing 10 best trials
Trial summary
Hyperparameters:
modelchoice: 8
poolchoice1: True
poolchoice2: False
poolchoice3: False
avgpool: True
units1: 544
units2: 448
units3: 128
unitsD1: 512
unitsD2: 128
dense2: False
dropout1: False
dropout2: False
dropout3: False
dropout4: True
lr: 0.0009250588597236514
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0203
Score: 0.7304526567459106
Trial summary
Hyperparameters:
modelchoice: 8
poolchoice1: True
poolchoice2: True
poolchoice3: False
avgpool: True
units1: 704
units2: 192
units3: 128
unitsD1: 608
unitsD2: 96
dense2: False
dropout1: False
dropout2: False
dropout3: False
dropout4: False
lr: 0.000911249456316419
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0239
Score: 0.7139917612075806
Trial summary
Hyperparameters:
modelchoice: 9
poolchoice1: True
poolchoice2: False
poolchoice3: True
avgpool: True
units1: 160
units2: 

In [ ]:
import shutil
best_model.save('bestmodel')
shutil.make_archive('/kaggle/working/bestmodel',
                    'zip',
                    '/kaggle/working/',
                    'bestmodel')

2022-12-16 22:53:03.000560: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


'/kaggle/working/bestmodel.zip'

In [ ]:
def best_model(input_shape, classes,
                  #  modelchoice=8,
                  #  units1=512,
                  #  units2=512,
                  # #  units3,
                  #  unitsD1=512,
                  # #  unitsD2,
                  #  dense2=False,
                  #  dropout1=False,
                  #  dropout2=False,
                  #  dropout3=False,
                  #  dropout4=True,
                  #  poolchoice1=True,
                  #  poolchoice2=False,
                  #  poolchoice3=False,
                  #  avgpool=True,
                  #  lr=.0009250588597236514
               ):
  
    # Build the neural network layer by layer
    model = tfk.models.Sequential()
    model.add(tfkl.Input(shape=input_shape, name='Input'))
    model.add(tfkl.Conv1D(units1, 3, padding='same', activation='relu'))
    model.add(tfkl.MaxPooling1D())
    model.add(tfkl.Conv1D(units2, 3, padding='same', activation='relu'))
    model.add(tfkl.GlobalAveragePooling1D())
    model.add(tfkl.Dropout(.3, seed=seed))

    # Classifier
    model.add(tfkl.Dense(unitsD1, activation='relu'))

    # Output
    model.add(tfkl.Flatten())
    model.add(tfkl.Dense(classes, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=tfk.optimizers.Adam(learning_rate=.0009250588597236514),
                  metrics='accuracy')

    # Return the model
    return model